In [1]:
'''
    ModelNet dataset. Support ModelNet40, ModelNet10, XYZ and normal channels. Up to 10000 points.
'''

import os
import os.path
import json
import numpy as np
import sys

def pc_normalize(pc):
    l = pc.shape[0]
    centroid = np.mean(pc, axis=0)
    pc = pc - centroid
    m = np.max(np.sqrt(np.sum(pc**2, axis=1)))
    pc = pc / m
    return pc

class ModelNetDataset():
    def __init__(self, root, batch_size = 32, npoints = 1024, split='train', normalize=True, normal_channel=True, modelnet10=False, cache_size=15000, shuffle=None):
        self.root = root
        self.batch_size = batch_size
        self.npoints = npoints
        self.normalize = normalize
        if modelnet10:
            self.catfile = os.path.join(self.root, 'modelnet10_shape_names.txt')
        else:
            self.catfile = os.path.join(self.root, 'modelnet40_shape_names.txt')
        self.cat = [line.rstrip() for line in open(self.catfile)]
        self.classes = dict(zip(self.cat, range(len(self.cat))))  
        self.normal_channel = normal_channel
        
        shape_ids = {}
        if modelnet10:
            shape_ids['train'] = [line.rstrip() for line in open(os.path.join(self.root, 'modelnet10_train.txt'))] 
            shape_ids['test']= [line.rstrip() for line in open(os.path.join(self.root, 'modelnet10_test.txt'))]
        else:
            shape_ids['train'] = [line.rstrip() for line in open(os.path.join(self.root, 'modelnet40_train.txt'))] 
            shape_ids['test']= [line.rstrip() for line in open(os.path.join(self.root, 'modelnet40_test.txt'))]
        assert(split=='train' or split=='test')
        shape_names = ['_'.join(x.split('_')[0:-1]) for x in shape_ids[split]]
        # list of (shape_name, shape_txt_file_path) tuple
        self.datapath = [(shape_names[i], os.path.join(self.root, shape_names[i], shape_ids[split][i])+'.txt') for i in range(len(shape_ids[split]))]

        self.cache_size = cache_size # how many data points to cache in memory
        self.cache = {} # from index to (point_set, cls) tuple

        if shuffle is None:
            if split == 'train': self.shuffle = True
            else: self.shuffle = False
        else:
            self.shuffle = shuffle

        self.reset()

    def _augment_batch_data(self, batch_data):
        if self.normal_channel:
            rotated_data = provider.rotate_point_cloud_with_normal(batch_data)
            rotated_data = provider.rotate_perturbation_point_cloud_with_normal(rotated_data)
        else:
            rotated_data = provider.rotate_point_cloud(batch_data)
            rotated_data = provider.rotate_perturbation_point_cloud(rotated_data)
    
        jittered_data = provider.random_scale_point_cloud(rotated_data[:,:,0:3])
        jittered_data = provider.shift_point_cloud(jittered_data)
        jittered_data = provider.jitter_point_cloud(jittered_data)
        rotated_data[:,:,0:3] = jittered_data
        return provider.shuffle_points(rotated_data)


    def _get_item(self, index): 
        if index in self.cache:
            point_set, cls = self.cache[index]
        else:
            fn = self.datapath[index]
            cls = self.classes[self.datapath[index][0]]
            cls = np.array([cls]).astype(np.int32)
            point_set = np.loadtxt(fn[1],delimiter=',').astype(np.float32)
            # Take the first npoints
            point_set = point_set[0:self.npoints,:]
            if self.normalize:
                point_set[:,0:3] = pc_normalize(point_set[:,0:3])
            if not self.normal_channel:
                point_set = point_set[:,0:3]
            if len(self.cache) < self.cache_size:
                self.cache[index] = (point_set, cls)
        return point_set, cls
        
    def __getitem__(self, index):
        return self._get_item(index)

    def __len__(self):
        return len(self.datapath)

    def num_channel(self):
        if self.normal_channel:
            return 6
        else:
            return 3

    def reset(self):
        self.idxs = np.arange(0, len(self.datapath))
        if self.shuffle:
            np.random.shuffle(self.idxs)
        self.num_batches = (len(self.datapath)+self.batch_size-1) // self.batch_size
        self.batch_idx = 0

    def has_next_batch(self):
        return self.batch_idx < self.num_batches

    def next_batch(self, augment=False):
        ''' returned dimension may be smaller than self.batch_size '''
        start_idx = self.batch_idx * self.batch_size
        end_idx = min((self.batch_idx+1) * self.batch_size, len(self.datapath))
        bsize = end_idx - start_idx
        batch_data = np.zeros((bsize, self.npoints, self.num_channel()))
        batch_label = np.zeros((bsize), dtype=np.int32)
        for i in range(bsize):
            ps,cls = self._get_item(self.idxs[i+start_idx])
            batch_data[i] = ps
            batch_label[i] = cls
        self.batch_idx += 1
        if augment: batch_data = self._augment_batch_data(batch_data)
        return batch_data, batch_label

In [3]:
from tqdm import tqdm

d = ModelNetDataset("/home/alex/Alex_documents/RGCNN/data/modelnet40_normal_resampled/",split="train",normal_channel=True)
train_data = np.zeros((len(d),1024,6))
train_label = np.zeros((len(d)))
print(train_data.shape)
for i in tqdm(range(len(d))):
    pc, label = d[i]
    train_data[i] = pc
    train_label[i] = label
print(train_data.shape)

FileNotFoundError: [Errno 2] No such file or directory: '/home/alex/Alex_documents/RGCNN/data/modelnet40_normal_resampled/modelnet40_shape_names.txt'

In [5]:
d = ModelNetDataset("/home/victor/workspace/thesis_ws/data/modelnet40_normal_resampled/",split="test",normal_channel=True)
test_data = np.zeros((len(d),1024,6))
test_label = np.zeros((len(d)))
print(test_data.shape)
for i in tqdm(range(len(d))):
    pc,label = d[i]
    test_data[i] = pc
    test_label[i] = label
print(test_data.shape)


  0%|          | 1/2468 [00:00<04:15,  9.65it/s]

(2468, 1024, 6)


100%|██████████| 2468/2468 [02:37<00:00, 15.67it/s]

(2468, 1024, 6)


In [6]:
np.save("cls_data_train.npy",train_data)
np.save("cls_label_train.npy",train_label)
np.save("cls_data_test.npy",test_data)
np.save("cls_label_test.npy",test_label)

In [ ]:
%pip install tqdm

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


### Preparing segmentation data

In [7]:
""" Original Author: Haoqiang Fan """
import numpy as np
import ctypes as ct
import cv2
import sys
import os
#BASE_DIR = os.path.dirname(os.path.abspath(__file__))
showsz=800
mousex,mousey=0.5,0.5
zoom=1.0
changed=True
def onmouse(*args):
    global mousex,mousey,changed
    y=args[1]
    x=args[2]
    mousex=x/float(showsz)
    mousey=y/float(showsz)
    changed=True
cv2.namedWindow('show3d')
cv2.moveWindow('show3d',0,0)
cv2.setMouseCallback('show3d',onmouse)

dll=np.ctypeslib.load_library(os.path.join(BASE_DIR, 'render_balls_so'),'.')

def showpoints(xyz,c_gt=None, c_pred = None ,waittime=0,showrot=False,magnifyBlue=0,freezerot=False,background=(0,0,0),normalizecolor=True,ballradius=10):
    global showsz,mousex,mousey,zoom,changed
    xyz=xyz-xyz.mean(axis=0)
    radius=((xyz**2).sum(axis=-1)**0.5).max()
    xyz/=(radius*2.2)/showsz
    if c_gt is None:
        c0=np.zeros((len(xyz),),dtype='float32')+255
        c1=np.zeros((len(xyz),),dtype='float32')+255
        c2=np.zeros((len(xyz),),dtype='float32')+255
    else:
        c0=c_gt[:,0]
        c1=c_gt[:,1]
        c2=c_gt[:,2]


    if normalizecolor:
        c0/=(c0.max()+1e-14)/255.0
        c1/=(c1.max()+1e-14)/255.0
        c2/=(c2.max()+1e-14)/255.0


    c0=np.require(c0,'float32','C')
    c1=np.require(c1,'float32','C')
    c2=np.require(c2,'float32','C')

    show=np.zeros((showsz,showsz,3),dtype='uint8')
    def render():
        rotmat=np.eye(3)
        if not freezerot:
            xangle=(mousey-0.5)*np.pi*1.2
        else:
            xangle=0
        rotmat=rotmat.dot(np.array([
            [1.0,0.0,0.0],
            [0.0,np.cos(xangle),-np.sin(xangle)],
            [0.0,np.sin(xangle),np.cos(xangle)],
            ]))
        if not freezerot:
            yangle=(mousex-0.5)*np.pi*1.2
        else:
            yangle=0
        rotmat=rotmat.dot(np.array([
            [np.cos(yangle),0.0,-np.sin(yangle)],
            [0.0,1.0,0.0],
            [np.sin(yangle),0.0,np.cos(yangle)],
            ]))
        rotmat*=zoom
        nxyz=xyz.dot(rotmat)+[showsz/2,showsz/2,0]

        ixyz=nxyz.astype('int32')
        show[:]=background
        dll.render_ball(
            ct.c_int(show.shape[0]),
            ct.c_int(show.shape[1]),
            show.ctypes.data_as(ct.c_void_p),
            ct.c_int(ixyz.shape[0]),
            ixyz.ctypes.data_as(ct.c_void_p),
            c0.ctypes.data_as(ct.c_void_p),
            c1.ctypes.data_as(ct.c_void_p),
            c2.ctypes.data_as(ct.c_void_p),
            ct.c_int(ballradius)
        )

        if magnifyBlue>0:
            show[:,:,0]=np.maximum(show[:,:,0],np.roll(show[:,:,0],1,axis=0))
            if magnifyBlue>=2:
                show[:,:,0]=np.maximum(show[:,:,0],np.roll(show[:,:,0],-1,axis=0))
            show[:,:,0]=np.maximum(show[:,:,0],np.roll(show[:,:,0],1,axis=1))
            if magnifyBlue>=2:
                show[:,:,0]=np.maximum(show[:,:,0],np.roll(show[:,:,0],-1,axis=1))
        if showrot:
            cv2.putText(show,'xangle %d'%(int(xangle/np.pi*180)),(30,showsz-30),0,0.5,cv2.cv.CV_RGB(255,0,0))
            cv2.putText(show,'yangle %d'%(int(yangle/np.pi*180)),(30,showsz-50),0,0.5,cv2.cv.CV_RGB(255,0,0))
            cv2.putText(show,'zoom %d%%'%(int(zoom*100)),(30,showsz-70),0,0.5,cv2.cv.CV_RGB(255,0,0))
    changed=True
    while True:
        if changed:
            render()
            changed=False
        cv2.imshow('show3d',show)
        if waittime==0:
            cmd=cv2.waitKey(10)%256
        else:
            cmd=cv2.waitKey(waittime)%256
        if cmd==ord('q'):
            break
        elif cmd==ord('Q'):
            sys.exit(0)

        if cmd==ord('t') or cmd == ord('p'):
            if cmd == ord('t'):
                if c_gt is None:
                    c0=np.zeros((len(xyz),),dtype='float32')+255
                    c1=np.zeros((len(xyz),),dtype='float32')+255
                    c2=np.zeros((len(xyz),),dtype='float32')+255
                else:
                    c0=c_gt[:,0]
                    c1=c_gt[:,1]
                    c2=c_gt[:,2]
            else:
                if c_pred is None:
                    c0=np.zeros((len(xyz),),dtype='float32')+255
                    c1=np.zeros((len(xyz),),dtype='float32')+255
                    c2=np.zeros((len(xyz),),dtype='float32')+255
                else:
                    c0=c_pred[:,0]
                    c1=c_pred[:,1]
                    c2=c_pred[:,2]
            if normalizecolor:
                c0/=(c0.max()+1e-14)/255.0
                c1/=(c1.max()+1e-14)/255.0
                c2/=(c2.max()+1e-14)/255.0
            c0=np.require(c0,'float32','C')
            c1=np.require(c1,'float32','C')
            c2=np.require(c2,'float32','C')
            changed = True



        if cmd==ord('n'):
            zoom*=1.1
            changed=True
        elif cmd==ord('m'):
            zoom/=1.1
            changed=True
        elif cmd==ord('r'):
            zoom=1.0
            changed=True
        elif cmd==ord('s'):
            cv2.imwrite('show3d.png',show)
        if waittime!=0:
            break
    return cmd

NameError: name 'BASE_DIR' is not defined

In [4]:

import os
import os.path
import json
import numpy as np
import sys

def pc_normalize(pc):
    l = pc.shape[0]
    centroid = np.mean(pc, axis=0)
    pc = pc - centroid
    m = np.max(np.sqrt(np.sum(pc**2, axis=1)))
    pc = pc / m
    return pc

class PartNormalDataset():
    def __init__(self, root, npoints = 2500, classification = False, split='train', normalize=True, return_cls_label = False):
        self.npoints = npoints
        self.root = root
        self.catfile = os.path.join(self.root, 'synsetoffset2category.txt')
        self.cat = {}
        
        self.classification = classification
        self.normalize = normalize
        self.return_cls_label = return_cls_label
        
        with open(self.catfile, 'r') as f:
            for line in f:
                ls = line.strip().split()
                self.cat[ls[0]] = ls[1]
        self.cat = {k:v for k,v in self.cat.items()}
        #print(self.cat)
            
        self.meta = {}
        with open(os.path.join(self.root, 'train_test_split', 'shuffled_train_file_list.json'), 'r') as f:
            train_ids = set([str(d.split('/')[2]) for d in json.load(f)])
        with open(os.path.join(self.root, 'train_test_split', 'shuffled_val_file_list.json'), 'r') as f:
            val_ids = set([str(d.split('/')[2]) for d in json.load(f)])
        with open(os.path.join(self.root, 'train_test_split', 'shuffled_test_file_list.json'), 'r') as f:
            test_ids = set([str(d.split('/')[2]) for d in json.load(f)])
        for item in self.cat:
            #print('category', item)
            self.meta[item] = []
            dir_point = os.path.join(self.root, self.cat[item])
            fns = sorted(os.listdir(dir_point))
            #print(fns[0][0:-4])
            if split=='trainval':
                fns = [fn for fn in fns if ((fn[0:-4] in train_ids) or (fn[0:-4] in val_ids))]
            elif split=='train':
                fns = [fn for fn in fns if fn[0:-4] in train_ids]
            elif split=='val':
                fns = [fn for fn in fns if fn[0:-4] in val_ids]
            elif split=='test':
                fns = [fn for fn in fns if fn[0:-4] in test_ids]
            else:
                print('Unknown split: %s. Exiting..'%(split))
                exit(-1)
                
            #print(os.path.basename(fns))
            for fn in fns:
                token = (os.path.splitext(os.path.basename(fn))[0]) 
                self.meta[item].append(os.path.join(dir_point, token + '.txt'))
        
        self.datapath = []
        for item in self.cat:
            for fn in self.meta[item]:
                self.datapath.append((item, fn))
            
         
        self.classes = dict(zip(self.cat, range(len(self.cat))))  
        # Mapping from category ('Chair') to a list of int [10,11,12,13] as segmentation labels
        self.seg_classes = {'Earphone': [16, 17, 18], 'Motorbike': [30, 31, 32, 33, 34, 35], 'Rocket': [41, 42, 43], 'Car': [8, 9, 10, 11], 'Laptop': [28, 29], 'Cap': [6, 7], 'Skateboard': [44, 45, 46], 'Mug': [36, 37], 'Guitar': [19, 20, 21], 'Bag': [4, 5], 'Lamp': [24, 25, 26, 27], 'Table': [47, 48, 49], 'Airplane': [0, 1, 2, 3], 'Pistol': [38, 39, 40], 'Chair': [12, 13, 14, 15], 'Knife': [22, 23]}

        for cat in sorted(self.seg_classes.keys()):
            print(cat, self.seg_classes[cat])
        
        self.cache = {} # from index to (point_set, cls, seg) tuple
        self.cache_size = 20000
        
    def __getitem__(self, index):
        if index in self.cache:
            point_set, normal, seg, cls = self.cache[index]
        else:
            fn = self.datapath[index]
            cat = self.datapath[index][0]
            cls = self.classes[cat]
            cls = np.array([cls]).astype(np.int32)
            data = np.loadtxt(fn[1]).astype(np.float32)
            point_set = data[:,0:3]
            if self.normalize:
                point_set = pc_normalize(point_set)
            normal = data[:,3:6]
            seg = data[:,-1].astype(np.int32)
            if len(self.cache) < self.cache_size:
                self.cache[index] = (point_set, normal, seg, cls)
                
        
        choice = np.random.choice(len(seg), self.npoints, replace=True)
        #resample
        point_set = point_set[choice, :]
        seg = seg[choice]
        normal = normal[choice,:]
        if self.classification:
            return point_set, normal, cls
        else:
            if self.return_cls_label:
                return point_set, normal, seg, cls
            else:
                return point_set, normal, seg
        
    def __len__(self):
        return len(self.datapath)



In [18]:
if __name__ == '__main__':
    d = PartNormalDataset(root = '../data/shapenetcore_partanno_segmentation_benchmark_v0_normal', split='trainval', npoints=1024)
    print(len(d))

    i = 500
    ps, normal, seg = d[i]
    print (d.datapath[i])
    print (np.max(seg), np.min(seg))
    print (ps.shape, seg.shape, normal.shape)
    print (ps)
    print (normal)
    
    sys.path.append('../utils')
    
    # showpoints(ps, normal+1, ballradius=8)

    d = PartNormalDataset(root = '../data/shapenetcore_partanno_segmentation_benchmark_v0_normal', classification = True)
    print(len(d))
    ps, normal, cls = d[0]
    print(ps.shape, type(ps), cls.shape,type(cls))

Airplane [0, 1, 2, 3]
Bag [4, 5]
Cap [6, 7]
Car [8, 9, 10, 11]
Chair [12, 13, 14, 15]
Earphone [16, 17, 18]
Guitar [19, 20, 21]
Knife [22, 23]
Lamp [24, 25, 26, 27]
Laptop [28, 29]
Motorbike [30, 31, 32, 33, 34, 35]
Mug [36, 37]
Pistol [38, 39, 40]
Rocket [41, 42, 43]
Skateboard [44, 45, 46]
Table [47, 48, 49]
13998
('Airplane', '../data/shapenetcore_partanno_segmentation_benchmark_v0_normal/02691156/3fe365251b54087af0478431b5ad57db.txt')
3 0
(1024, 3) (1024,) (1024, 3)
[[ 0.09738774  0.00546969 -0.8964166 ]
 [ 0.16769218 -0.00677618 -0.41418064]
 [ 0.01516543 -0.08585501  0.74770147]
 ...
 [ 0.2265981  -0.01117705  0.32111856]
 [-0.5533657   0.10272599  0.0414042 ]
 [ 0.19956425  0.01574748  0.09921676]]
[[ 0.0325    0.9995    0.003013]
 [-0.09137   0.9957    0.01664 ]
 [-0.1217    0.7157    0.6878  ]
 ...
 [ 0.01265   0.9997    0.02266 ]
 [ 0.01154   0.9373    0.3483  ]
 [-0.01631   0.09668   0.9952  ]]
Airplane [0, 1, 2, 3]
Bag [4, 5]
Cap [6, 7]
Car [8, 9, 10, 11]
Chair [12, 13, 14,

In [24]:
from tqdm import tqdm
train_pos = np.zeros([len(d), 2048, 3])
train_seg = np.zeros([len(d), 2048])
train_norm = np.zeros([len(d), 2048, 3])
d = PartNormalDataset(root = '../data/shapenetcore_partanno_segmentation_benchmark_v0_normal', split='train', npoints=2048)

for i in tqdm(range(len(d))):
    ps, normal, seg = d[i]
    train_pos[i] = ps
    train_seg[i] = seg
    train_norm[i] = normal
print(f"Pos: {train_pos.shape}")

  0%|          | 7/12137 [00:00<03:18, 61.04it/s]

Airplane [0, 1, 2, 3]
Bag [4, 5]
Cap [6, 7]
Car [8, 9, 10, 11]
Chair [12, 13, 14, 15]
Earphone [16, 17, 18]
Guitar [19, 20, 21]
Knife [22, 23]
Lamp [24, 25, 26, 27]
Laptop [28, 29]
Motorbike [30, 31, 32, 33, 34, 35]
Mug [36, 37]
Pistol [38, 39, 40]
Rocket [41, 42, 43]
Skateboard [44, 45, 46]
Table [47, 48, 49]


100%|██████████| 12137/12137 [03:26<00:00, 58.78it/s]

Pos: (13998, 2048, 3)


In [25]:
np.save("seg_train_pos.npy", train_pos)
np.save("seg_train_seg.npy", train_seg)
np.save("seg_train_norm.npy", train_norm)

In [26]:
from tqdm import tqdm
val_pos = np.zeros([len(d), 2048, 3])
val_seg = np.zeros([len(d), 2048])
val_norm = np.zeros([len(d), 2048, 3])
d = PartNormalDataset(root = '../data/shapenetcore_partanno_segmentation_benchmark_v0_normal', split='val', npoints=2048)

for i in tqdm(range(len(d))):
    ps, normal, seg = d[i]
    val_pos[i] = ps
    val_seg[i] = seg
    val_norm[i] = normal
print(f"Pos: {val_pos.shape}")

  0%|          | 6/1870 [00:00<00:34, 54.72it/s]

Airplane [0, 1, 2, 3]
Bag [4, 5]
Cap [6, 7]
Car [8, 9, 10, 11]
Chair [12, 13, 14, 15]
Earphone [16, 17, 18]
Guitar [19, 20, 21]
Knife [22, 23]
Lamp [24, 25, 26, 27]
Laptop [28, 29]
Motorbike [30, 31, 32, 33, 34, 35]
Mug [36, 37]
Pistol [38, 39, 40]
Rocket [41, 42, 43]
Skateboard [44, 45, 46]
Table [47, 48, 49]


100%|██████████| 1870/1870 [00:32<00:00, 57.78it/s]

Pos: (12137, 2048, 3)


In [27]:
np.save("seg_val_pos.npy", val_pos)
np.save("seg_val_seg.npy", val_seg)
np.save("seg_val_norm.npy", val_norm)

In [29]:
from tqdm import tqdm
d = PartNormalDataset(root = '../data/shapenetcore_partanno_segmentation_benchmark_v0_normal', split='test', npoints=2048)
test_pos = np.zeros([len(d), 2048, 3])
test_seg = np.zeros([len(d), 2048])
test_norm = np.zeros([len(d), 2048, 3])

for i in tqdm(range(len(d))):
    ps, normal, seg = d[i]
    test_pos[i] = ps
    test_seg[i] = seg
    test_norm[i] = normal
print(f"Pos: {test_pos.shape}")

  0%|          | 6/2874 [00:00<00:50, 56.96it/s]

Airplane [0, 1, 2, 3]
Bag [4, 5]
Cap [6, 7]
Car [8, 9, 10, 11]
Chair [12, 13, 14, 15]
Earphone [16, 17, 18]
Guitar [19, 20, 21]
Knife [22, 23]
Lamp [24, 25, 26, 27]
Laptop [28, 29]
Motorbike [30, 31, 32, 33, 34, 35]
Mug [36, 37]
Pistol [38, 39, 40]
Rocket [41, 42, 43]
Skateboard [44, 45, 46]
Table [47, 48, 49]


100%|██████████| 2874/2874 [00:48<00:00, 58.73it/s]

Pos: (2874, 2048, 3)


In [30]:
np.save("seg_test_pos.npy", test_pos)
np.save("seg_test_seg.npy", test_seg)
np.save("seg_test_norm.npy", test_norm)